# UCI-HAR CNN

## Imports

In [51]:
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import test 
import train

## Load UCI-HAR dataset

In [52]:
def load_set(part: str):
    data = np.hstack([np.loadtxt(Path(part)/'Inertial Signals'/f'{sensor}_{axis}_{part}.txt')
                for sensor in ('body_acc', 'body_gyro', 'total_acc')
                    for axis in ('x', 'y', 'z')]).reshape((-1, 128, 9))
    labels = to_categorical(np.loadtxt(Path(part)/f'y_{part}.txt') - 1)
    return data, labels

x_train, y_train = load_set('train')
x_test, y_test = load_set('test')

## Export small dataset

In [53]:
x_test_250 = x_test[0:250]
y_test_250 = y_test[0:250]
np.savetxt('x_test_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model

In [54]:
num_samples = x_train.shape[0]
num_features = x_train.shape[1]
num_classes = y_train.shape[1]  # Use one-hot encoded shape
    
print(f'Loaded {num_samples} samples. Features: {num_features}, Classes: {num_classes}')

model = models.Sequential([
    layers.InputLayer(input_shape=(num_features, 9)),
    layers.Flatten(),  # Flatten 3D input to 1D
    layers.Dense(16, activation='relu'),  # Hidden layer 1
    layers.Dense(8, activation='relu'),   # Hidden layer 2 (helps with complexity)
    layers.Dense(num_classes, activation='softmax') # Output layer
])
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Loaded 7352 samples. Features: 128, Classes: 6


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_6 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 16)             │        18,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 6)              │            54 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,638 (72.80 KB)

 Trainable params: 18,638 (72.80 KB)

 Non-trainable params: 0 (0.00 B)

## Train model

In [55]:
model.fit(x_train, y_train, epochs=15, validation_data=(x_test_250, y_test_250))

Epoch 1/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.6153 - loss: 0.9663 - val_categorical_accuracy: 0.5920 - val_loss: 1.0558
Epoch 2/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.6153 - loss: 0.9663 - val_categorical_accuracy: 0.5920 - val_loss: 1.0558
Epoch 2/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - categorical_accuracy: 0.7871 - loss: 0.5663 - val_categorical_accuracy: 0.8160 - val_loss: 0.4304
Epoch 3/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - categorical_accuracy: 0.7871 - loss: 0.5663 - val_categorical_accuracy: 0.8160 - val_loss: 0.4304
Epoch 3/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - categorical_accuracy: 0.8327 - loss: 0.4449 - val_categorical_accuracy: 0.7320 - val_loss: 0.6236
Epoch 4/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - categorical_accuracy: 0.8327 - loss: 0.4449 - val_categorical_accuracy: 0.7320 - val_loss: 0.6236
Epoch 4/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - categorical_accuracy: 0.

## Evaluate model on small dataset

In [56]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test.argmax(axis=1)))

8/8 - 0s - 3ms/step - categorical_accuracy: 0.6640 - loss: 0.9802
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
tf.Tensor(
[[32 18  0  0  3  0]
 [ 0 25  0  0  0  0]
 [ 1  5 18  0  0  0]
 [ 0  0  3 11 32  0]
 [ 0 22  0  0 32  0]
 [ 0  0  0  0  0 48]], shape=(6, 6), dtype=int32)
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
tf.Tensor(
[[32 18  0  0  3  0]
 [ 0 25  0  0  0  0]
 [ 1  5 18  0  0  0]
 [ 0  0  3 11 32  0]
 [ 0 22  0  0 32  0]
 [ 0  0  0  0  0 48]], shape=(6, 6), dtype=int32)


## Evaluate model on complete dataset

In [57]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

93/93 - 0s - 746us/step - categorical_accuracy: 0.7974 - loss: 1.3102
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step
tf.Tensor(
[[355  65  47   6  23   0]
 [ 43 370  49   0   9   0]
 [ 16  25 371   3   5   0]
 [  3  22   8 389  69   0]
 [  8  58   3 108 355   0]
 [  0  18   9   0   0 510]], shape=(6, 6), dtype=int32)
tf.Tensor(
[[355  65  47   6  23   0]
 [ 43 370  49   0   9   0]
 [ 16  25 371   3   5   0]
 [  3  22   8 389  69   0]
 [  8  58   3 108 355   0]
 [  0  18   9   0   0 510]], shape=(6, 6), dtype=int32)


## Save trained model

In [58]:
model.save('uci-har_si4.h5')

## Remove SoftMax layer

In [59]:
# CORRECT way to remove Softmax but KEEP the layer weights
model_without_softmax = tf.keras.models.clone_model(model)
model_without_softmax.set_weights(model.get_weights())

# Swap the last layer's activation from 'softmax' to 'linear' (which is nothing)
model_without_softmax.layers[-1].activation = tf.keras.activations.linear

# Save and Summary
model_without_softmax.save('uci-har_si4_no_softmax.h5')
model_without_softmax.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_6 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 16)             │        18,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 6)              │            54 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,278 (145.62 KB)

 Trainable params: 18,638 (72.80 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,640 (72.82 KB)

## Generate C for the trained model

In [60]:
# ---------------------------------------------------------
# CORRECTED C GENERATION SCRIPT (Fixed-Point + Transpose)
# ---------------------------------------------------------
import numpy as np

# 1. Config
FIXED_POINT = 9
SCALE = 2 ** FIXED_POINT

c_code = """
#ifndef MODEL_H
#define MODEL_H
#include <stdint.h>

#define FIXED_POINT 9
#define MODEL_INPUT_SAMPLES 128
#define MODEL_INPUT_CHANNELS 9
#define MODEL_INPUT_TOTAL 1152 
#define MODEL_OUTPUT_SAMPLES 6

typedef int16_t number_t;
typedef int32_t long_number_t;

number_t clamp_to_number_t(long_number_t value) {
    if (value > 32767) return 32767;
    if (value < -32768) return -32768;
    return (number_t)value;
}

number_t activation_relu(number_t value) {
    if (value < 0) return 0;
    return value;
}
"""

# 2. Extract and Convert Weights
# We expect 3 Dense layers: Dense(16) -> Dense(8) -> Dense(6)
layers = [l for l in model_without_softmax.layers if 'dense' in l.name]

for i, layer in enumerate(layers):
    w, b = layer.get_weights()
    
    # TRANSPOSE W: Convert from [Input][Output] -> [Output][Input]
    # This makes the C code loop much faster and simpler
    w = w.T 
    
    # QUANTIZE: Scale by 512 (Fixed Point 9)
    w_int = (w * SCALE).astype(np.int16)
    b_int = (b * SCALE).astype(np.int16)
    
    # Write Weights
    flat_w = w_int.flatten()
    c_code += f'\n// Layer {i} Weights ({w.shape})\n'
    c_code += f'const int16_t weights_{i*2}[] = {{\n'
    for j, val in enumerate(flat_w):
        c_code += f'{val}, '
        if (j+1) % 16 == 0: c_code += '\n'
    c_code += '};\n'
    
    # Write Biases
    c_code += f'\n// Layer {i} Biases\n'
    c_code += f'const int16_t weights_{i*2+1}[] = {{\n'
    for val in b_int:
        c_code += f'{val}, '
    c_code += '};\n'

# 3. Write Inference Function
c_code += """
// Temporary buffers
number_t layer1_out[16]; 
number_t layer2_out[8];

void cnn(number_t* input, number_t* output) {
    // --- LAYER 1: Dense 16 ---
    for (int i = 0; i < 16; i++) { 
        long_number_t acc = 0; 
        for (int j = 0; j < 1152; j++) { 
            acc += (long_number_t)input[j] * weights_0[i * 1152 + j];
        }
        acc = acc >> FIXED_POINT;
        acc += weights_1[i];
        layer1_out[i] = activation_relu(clamp_to_number_t(acc));
    }

    // --- LAYER 2: Dense 8 ---
    for (int i = 0; i < 8; i++) { 
        long_number_t acc = 0;
        for (int j = 0; j < 16; j++) { 
            acc += (long_number_t)layer1_out[j] * weights_2[i * 16 + j];
        }
        acc = acc >> FIXED_POINT;
        acc += weights_3[i];
        layer2_out[i] = activation_relu(clamp_to_number_t(acc));
    }

    // --- LAYER 3: Dense 6 (Output) ---
    for (int i = 0; i < 6; i++) { 
        long_number_t acc = 0;
        for (int j = 0; j < 8; j++) { 
            acc += (long_number_t)layer2_out[j] * weights_4[i * 8 + j];
        }
        acc = acc >> FIXED_POINT;
        acc += weights_5[i];
        output[i] = clamp_to_number_t(acc);
    }
}
#endif
"""

with open('model.h', 'w') as f:
    f.write(c_code)

print("Corrected model.h generated!")

Corrected model.h generated!
